# 16: Cost by Income

Another approach to the inclusion of trip maker or context characteristics is through interactions with mode attributes. The most common example of this approach is to take account of the expectation that low-income travelers will be more sensitive to travel cost than high-income travelers by using cost divided by income in place of cost as an explanatory variable. Such a specification implies that the importance of cost in mode choice diminishes with increasing household income.

Model 16 drops travel cost to include travel cost divided by income. (pp. 125)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import P, X

for a in [2, 3]:
    m.utility_co[a] = +P("hhinc#2,3") * X("hhinc")
for a in [4, 5, 6]:
    m.utility_co[a] = +P(f"hhinc#{a}") * X("hhinc")

Sometimes we may want to define a part of the utility function that is common across all (or almost all) of the alternatives. We can access a dictionary (more generically called a “mapping”) of alternative codes to alternative names, which can be found via the Dataset.dc.alts_mapping attribute:

In [ ]:
d.dc.alts_mapping

Using this like a standard Python dictionary, we can iterate over all the alternatives, skipping 1, and setting alternative specific constants (ASC’s) for the rest.

In [ ]:
for a, name in d.dc.alts_mapping.items():
    if a == 1:
        continue
    m.utility_co[a] += (
        +P("ASC_" + name)
        + P("vehbywrk_" + name) * X("vehbywrk")
        + P("wkcbd_" + name) * X("wkccbd + wknccbd")
        + P("wkempden_" + name) * X("wkempden")
    )

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +P("nonmotorized_time") * X("(altid> 4) * tottime")
    + P("motorized_time") * X("(altid <= 4) * ivtt")
    + (P("motorized_time") + (P("motorized_ovtbydist") / X("dist")))
    * X("(altid <= 4) * ovtt")
    + P("costbyinc") * X("totcost/hhinc")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 16, Cost by Income"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, method="bhhh")
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "Ownership",
        "vehbywrk.*",
    ),
    (
        "Zonal",
        "wkcbd.*",
        "wkempden.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_Bike": -1.6217774612007492,
    "ASC_SR2": -1.7297986582095874,
    "ASC_SR3+": -3.6562561506975255,
    "ASC_Transit": -0.6917042404795228,
    "ASC_Walk": 0.07521549340170937,
    "costbyinc": -0.05177365897066612,
    "hhinc#2,3": 3.691866127922073e-05,
    "hhinc#4": -0.005335573957018,
    "hhinc#5": -0.008671987053899464,
    "hhinc#6": -0.006017166395409271,
    "motorized_ovtbydist": -0.13272166895031992,
    "motorized_time": -0.0201578217327816,
    "nonmotorized_time": -0.045438649548417406,
    "vehbywrk_Bike": -0.7040646431527947,
    "vehbywrk_SR2": -0.38161738059345535,
    "vehbywrk_SR3+": -0.13880487622153995,
    "vehbywrk_Transit": -0.937505306213354,
    "vehbywrk_Walk": -0.72385335687274,
    "wkcbd_Bike": 0.4863239419169535,
    "wkcbd_SR2": 0.24714212608546096,
    "wkcbd_SR3+": 1.0943587942374322,
    "wkcbd_Transit": 1.3056155684640574,
    "wkcbd_Walk": 0.09724802958992065,
    "wkempden_Bike": 0.0019224683185795574,
    "wkempden_SR2": 0.0015964358018843575,
    "wkempden_SR3+": 0.0022037880176673103,
    "wkempden_Transit": 0.0031316557035939658,
    "wkempden_Walk": 0.002881429335761498,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )